In [1]:
import numpy as np
import pandas as pd
import holidays
import optuna

from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split, cross_val_score, KFold 
from sklearn.metrics import mean_absolute_percentage_error, make_scorer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [2]:
df = pd.read_csv('data/train.csv')
df_pred = pd.read_csv('data/test.csv')

In [3]:
df

,id,date,country,store,product,num_sold
0,0,2010-01-01,Canada,Discount Stickers,Holographic Goose,NaN
1,1,2010-01-01,Canada,Discount Stickers,Kaggle,973.0
2,2,2010-01-01,Canada,Discount Stickers,Kaggle Tiers,906.0
3,3,2010-01-01,Canada,Discount Stickers,Kerneler,423.0
4,4,2010-01-01,Canada,Discount Stickers,Kerneler Dark Mode,491.0
...,...,...,...,...,...,...
230125,230125,2016-12-31,Singapore,Premium Sticker Mart,Holographic Goose,466.0
230126,230126,2016-12-31,Singapore,Premium Sticker Mart,Kaggle,2907.0
230127,230127,2016-12-31,Singapore,Premium Sticker Mart,Kaggle Tiers,2299.0
230128,230128,2016-12-31,Singapore,Premium Sticker Mart,Kerneler,1242.0


In [4]:
# Привожу дату к типу pd.datetime
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
df_pred['date'] = pd.to_datetime(df_pred['date'], format='%Y-%m-%d')

In [5]:
# Новые признаки из даты
df['day'] = df.date.dt.day
df['month'] = df.date.dt.month
df['year'] = df.date.dt.year
# df['quarter'] = df.date.dt.quarter
# df['dayofyear'] = df.date.dt.dayofyear
# df['weekday'] = df.date.dt.weekday

df_pred['day'] = df_pred.date.dt.day
df_pred['month'] = df_pred.date.dt.month
df_pred['year'] = df_pred.date.dt.year
# df_pred['quarter'] = df_pred.date.dt.quarter
# df_pred['dayofyear'] = df_pred.date.dt.dayofyear
# df_pred['weekday'] = df_pred.date.dt.weekday

In [8]:
# Удаляю id
df.drop(columns='id', inplace=True)
df_pred.drop(columns='id', inplace=True)

# Удаляю Nan значения
df.dropna(subset=['num_sold'], inplace=True)

In [9]:
# удаляю date
df.drop(columns='date', inplace=True)
df_pred.drop(columns='date', inplace=True)

KeyError: "['date'] not found in axis"

In [10]:
def cat_val(df, column):
    uniq = df[column].unique()
    for i in uniq:
        df[i] = df[column].apply(lambda x: x == i)
    df.drop(columns=column, inplace=True)


for i in ['country', 'store', 'product']:
    cat_val(df, i)
    cat_val(df_pred, i)

In [12]:
X = df.drop(columns='num_sold')
y = df['num_sold']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
# def objective(trial: optuna.Trial):
#     '''
#     Обычная сигнатура функции оптимизации для optuna.
#     '''
#     param = {
#             "metric": 'MAPE',
#             "verbosity": -1,
#             "random_state": 42,
#             "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1, log=True),
#             "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
#             'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
#             'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
#             "max_depth": trial.suggest_int("max_depth", 5, 25),
# #            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.3, 0.9),
# #            "subsample": trial.suggest_float("subsample", 0.8, 1.0),
#             "min_child_samples": trial.suggest_int("min_child_samples", 5, 50),
#             'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
#             'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0)}
    
#     lgbm_model = LGBMRegressor(**param)
#     lgbm_model.fit(X_train, y_train, eval_set=(X_test, y_test))
#     y_pred = lgbm_model.predict(X_test)
#     return mean_absolute_percentage_error(y_test, y_pred)

# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=100)

[I 2025-01-19 15:31:46,528] A new study created in memory with name: no-name-0cd2129c-5945-4b8e-8240-95890482cc16
[I 2025-01-19 15:31:49,856] Trial 0 finished with value: 0.5027828467097253 and parameters: {'learning_rate': 0.04261862924223396, 'n_estimators': 302, 'lambda_l1': 1.3439355108973337, 'lambda_l2': 2.7287957037188385e-07, 'max_depth': 9, 'min_child_samples': 6, 'feature_fraction': 0.4173547702458782, 'bagging_fraction': 0.8865583370411187}. Best is trial 0 with value: 0.5027828467097253.
[I 2025-01-19 15:31:54,737] Trial 1 finished with value: 0.1773305523038653 and parameters: {'learning_rate': 0.07722046500570272, 'n_estimators': 502, 'lambda_l1': 1.1391594245993499e-07, 'lambda_l2': 0.000856645513222102, 'max_depth': 12, 'min_child_samples': 19, 'feature_fraction': 0.5148953469947037, 'bagging_fraction': 0.7273081550528255}. Best is trial 1 with value: 0.1773305523038653.
[I 2025-01-19 15:31:58,398] Trial 2 finished with value: 0.4069341465518046 and parameters: {'learni

In [16]:
# study.best_params

{'learning_rate': 0.04718722160252457,
 'n_estimators': 843,
 'lambda_l1': 0.00010775286482054532,
 'lambda_l2': 0.007551578014641231,
 'max_depth': 12,
 'min_child_samples': 6,
 'feature_fraction': 0.7957302298889435,
 'bagging_fraction': 0.5418003123219285}

In [17]:
# study_params = {
#     "metric": 'MAPE',
#     "verbosity": -1,
#     "random_state": 42,
#     'learning_rate': 0.04718722160252457,
#     'n_estimators': 843,
#     'lambda_l1': 0.00010775286482054532,
#     'lambda_l2': 0.007551578014641231,
#     'max_depth': 12,
#     'min_child_samples': 6,
#     'feature_fraction': 0.7957302298889435,
#     'bagging_fraction': 0.5418003123219285
# }

In [13]:
params = {
    "metric": 'MAPE',
    "verbosity": -1,
    "random_state": 42,
}

In [15]:
lgbmr_model = LGBMRegressor(**params)

lgbmr_model.fit(X_train, y_train, eval_set=(X_test, y_test))
y_pred = lgbmr_model.predict(X_test)
mae = mean_absolute_percentage_error(y_test, y_pred)
pred = lgbmr_model.predict(df_pred)
mae

0.16653113219269797

In [19]:
df_pred_duble = pd.read_csv('data/test.csv')
df_pred_Id = df_pred_duble['id']

output = pd.DataFrame({'id': df_pred_Id, 'num_sold': pred})
output.to_csv('data/lgbmr_model.csv', index=False)